In [ ]:
# Перезагружаем данные снова после сброса выполнения
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

# Загружаем файл
file_path = '../npz/BTCUSD_1T.npz'
data = np.load(file_path, allow_pickle=True)

# Проверяем содержимое
if 'data' in data:
    raw_data = data['data']
else:
    raise ValueError("Файл не содержит ключа 'data'")

# Преобразуем в DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
df = pd.DataFrame(raw_data, columns=columns)

# Преобразуем timestamp в datetime и делаем индексом
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Преобразуем числовые колонки в float
for col in ['open', 'high', 'low', 'close', 'volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Функция для построения сетки фазового пространства и заполнения кеша вероятностей
def build_phase_space_grid(df, window_short, window_long, tau, grid_size=500):
    """
    Разбивает фазовое пространство на сетку и заполняет кеш вероятностей переходов.
    
    :param df: DataFrame с ценами
    :param window_short: короткое окно сглаживания
    :param window_long: длинное окно сглаживания
    :param tau: шаг прогноза
    :param grid_size: количество ячеек на оси (по умолчанию 500x500)
    :return: кэш вероятностей { (i_x, i_y) -> { (i_x', i_y'): вероятность } }
    """
    polyorder = 3  # Порядок полинома

    # Применяем Savitzky-Golay фильтр для сглаживания
    smoothed_short = savgol_filter(df['close'].values, window_short, polyorder)
    smoothed_long = savgol_filter(df['close'].values, window_long, polyorder)

    # Вычисляем разность сглаженных цен в процентах от длинной средней
    tmp = (smoothed_short - smoothed_long) / smoothed_long
    price_diff = savgol_filter(tmp, window_short, polyorder, deriv=0)
    price_diff_deriv = savgol_filter(tmp, window_short, polyorder, deriv=1)

    # Определяем границы фазового пространства
    x_min, x_max = np.min(price_diff), np.max(price_diff)
    y_min, y_max = np.min(price_diff_deriv), np.max(price_diff_deriv)

    # Функция для преобразования координат в индексы сетки
    def to_grid(x, y):
        i_x = int((x - x_min) / (x_max - x_min) * (grid_size - 1))
        i_y = int((y - y_min) / (y_max - y_min) * (grid_size - 1))
        return (i_x, i_y)

    # Заполняем кеш вероятностей
    cache = {}  # { (i_x, i_y) -> { (i_x', i_y'): count } }
    num_points = len(price_diff) - tau

    for t in range(num_points):
        # Определяем текущую ячейку фазового пространства
        cell_now = to_grid(price_diff[t], price_diff_deriv[t])
        cell_future = to_grid(price_diff[t + tau], price_diff_deriv[t + tau])

        # Добавляем переход в кеш
        if cell_now not in cache:
            cache[cell_now] = {}
        if cell_future not in cache[cell_now]:
            cache[cell_now][cell_future] = 0

        cache[cell_now][cell_future] += 1

    # закомментирую, т.к. хочу видеть на карте не вероятности, а количество переходов, чтобы было понятно насколько
    # статистически значимый результат получился.
    # # Нормализация вероятностей
    # for cell_now in cache:
    #     total_transitions = sum(cache[cell_now].values())
    #     for cell_future in cache[cell_now]:
    #         cache[cell_now][cell_future] /= total_transitions  # Преобразуем в вероятность

    return cache, (x_min, x_max, y_min, y_max)

# # Строим кеш вероятностей для одного фазового портрета (30-60)
# cache_30_60, grid_bounds_30_60 = build_phase_space_grid(df, window_short=30, window_long=60, tau=10, grid_size=500)

# # Проверяем, сколько ячеек заполнено
# num_filled_cells = len(cache_30_60)
# print(f"Количество заполненных ячеек: {num_filled_cells}")


In [ ]:
# Функция для прогноза будущего положения цены и визуализации тепловой карты
def plot_price_forecast_with_heatmap(df, cache, grid_bounds, window_short, window_long, tau, num_points, grid_size):
    """
    Строит график цен и наносит прогноз вероятностей в виде тепловой карты.
    
    :param df: DataFrame с ценами
    :param cache: Кеш вероятностей переходов в фазовом пространстве
    :param grid_bounds: Границы фазового пространства (x_min, x_max, y_min, y_max)
    :param window_short: короткое окно сглаживания
    :param window_long: длинное окно сглаживания
    :param tau: количество шагов вперед для прогноза
    :param num_points: количество точек на графике для упрощённого отображения
    """
    polyorder = 3  # Порядок полинома
    x_min, x_max, y_min, y_max = grid_bounds

    # Применяем Savitzky-Golay фильтр для сглаживания
    smoothed_short = savgol_filter(df['close'].values, window_short, polyorder)
    smoothed_long = savgol_filter(df['close'].values, window_long, polyorder)

    # Вычисляем разность сглаженных цен в процентах от длинной средней
    tmp = (smoothed_short - smoothed_long) / smoothed_long
    price_diff = savgol_filter(tmp, window_short, polyorder, deriv=0)
    price_diff_deriv = savgol_filter(tmp, window_short, polyorder, deriv=1)

    # Функция для поиска ближайшей ячейки в фазовом пространстве
    def to_grid(x, y):
        i_x = int((x - x_min) / (x_max - x_min) * (grid_size - 1))
        i_y = int((y - y_min) / (y_max - y_min) * (grid_size - 1))
        return (i_x, i_y)

    # Собираем прогнозируемые точки
    future_points = []
    future_probabilities = []

    for t in range(num_points):
        # Определяем текущую ячейку фазового пространства
        cell_now = to_grid(price_diff[t], price_diff_deriv[t])

        # Получаем вероятности будущего положения
        if cell_now in cache:
            for cell_future, prob in cache[cell_now].items():
                # Получаем индекс будущей точки в данных
                future_index = t + tau

                # Если индекс не выходит за границы
                if future_index < len(df):
                    # Пересчитываем прогнозируемую цену обратно
                    smoothed_long = smoothed_long[t]  # Грубое предположение, что не меняется
                    future_price_diff = (cell_future[0] / (grid_size - 1)) * (x_max - x_min) + x_min
                    future_price = future_price_diff * smoothed_long + df['close'].iloc[t]

                    future_time = df.index[future_index]

                    # Добавляем точку на график
                    future_points.append((future_time, future_price))
                    future_probabilities.append(prob)

    # Построение графика
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Основной график цен (берём первые num_points значений)
    ax.plot(df.index[:num_points], df['close'].iloc[:num_points], label="Цена BTC/USD", color='black', alpha=0.7)

    # Наносим прогнозируемые точки
    if future_points:
        future_times, future_prices = zip(*future_points)
        scatter = ax.scatter(future_times, future_prices, c=future_probabilities, cmap='coolwarm', alpha=0.5, s=5)

        # Добавляем цветовую шкалу
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label("Вероятность")

    # Настройки графика
    ax.set_title(f"Прогноз на {tau} шагов вперёд (ФП {window_short}-{window_long})")
    ax.set_xlabel("Дата")
    ax.set_ylabel("Цена BTC/USD")
    ax.legend()

    plt.show()

# Запуск для одного фазового портрета (30-60)
# plot_price_forecast_with_heatmap(df, cache_30_60, grid_bounds_30_60, window_short=30, window_long=60, tau=10, num_points=1000, grid_size=200)


In [ ]:
%matplotlib widget

In [ ]:
# 1) Делим данные
N = len(df)
train_size = int(0.8 * N)
train_df = df.iloc[:train_size]
test_df = df.iloc[train_size:]

tau = 5
sh = 60
lg = 120
gs = 200
# 2) Строим кеш на train_df
cache_30_60, grid_bounds_30_60 = build_phase_space_grid(
    train_df, window_short=sh, window_long=lg, tau=tau, grid_size=gs
)

# 3) Визуализируем на test_df (первые 500 точек)
plot_price_forecast_with_heatmap(
    test_df, 
    cache_30_60, 
    grid_bounds_30_60, 
    window_short=sh, 
    window_long=lg, 
    tau=tau, 
    num_points=100,
    grid_size=gs
)
